In [3]:
# Load cleaned M-Pesa data from the Spark table
df = spark.read.table("mpesa_cleaned")  # Or use "mpesa_normalized" if that’s your table name
df.show(30)


StatementMeta(, 3f70d7a4-b60b-4600-b977-5175bc0acb75, 5, Finished, Available, Finished)

+----------+-----+--------------------+--------+---------+--------+--------------------+
|      date| time|    transaction_type|  amount|direction| balance|           narration|
+----------+-----+--------------------+--------+---------+--------+--------------------+
|2024-10-09|16:38|      Salary Payment|  3117.3|      Out| 11882.7|Salary Payment ex...|
|2024-10-09| 9:34|Paybill Payment -...| 2577.35|      Out| 9305.35|Paybill Payment -...|
|2024-10-09|10:20|    Customer Payment|12537.29|       In|21842.64|Customer Payment ...|
|2024-10-09| 8:46|      Loan Repayment|12538.96|      Out| 9303.68|Loan Repayment ex...|
|2024-10-09|18:58|    Customer Payment|11545.49|       In|20849.17|Customer Payment ...|
|2024-10-10|17:12|           Buy Goods| 5421.95|      Out|15427.22|  Buy Goods executed|
|2024-10-10|11:42|Utility Bill Payment| 4776.95|      Out|10650.27|Utility Bill Paym...|
|2024-10-10|17:34|Paybill Payment -...| 5373.64|      Out| 5276.63|Paybill Payment -...|
|2024-10-10|18:03|   

In [6]:
import pandas as pd

# Load to pandas for Prophet
df_pd = df.filter(df.direction == "In").select("date", "amount").groupBy("date").sum("amount").toPandas()
df_pd.columns = ["ds", "y"]  # Prophet expects columns as 'ds' and 'y'

df_pd['ds'] = pd.to_datetime(df_pd['ds'])
df_pd = df_pd.sort_values("ds")
df_pd.tail()


StatementMeta(, 3f70d7a4-b60b-4600-b977-5175bc0acb75, 8, Finished, Available, Finished)

,ds,y
78,2025-04-03,12714.08
15,2025-04-04,8722.63
3,2025-04-05,19228.03
23,2025-04-06,34332.75
123,2025-04-07,19056.02


In [7]:
from prophet import Prophet

model = Prophet()
model.fit(df_pd)

future = model.make_future_dataframe(periods=30)
forecast = model.predict(future)

forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]].tail(10)


StatementMeta(, 3f70d7a4-b60b-4600-b977-5175bc0acb75, 9, Finished, Available, Finished)

06:45:00 - cmdstanpy - INFO - Chain [1] start processing
06:45:00 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat,yhat_lower,yhat_upper
146,2025-04-28,21804.573583,4546.372282,37743.445079
147,2025-04-29,20178.704990,3956.744828,36581.289951
148,2025-04-30,22293.398894,5423.343097,39385.551472
149,2025-05-01,23792.887635,7026.482450,41643.897273
150,2025-05-02,25820.478400,8358.249921,42077.170871
151,2025-05-03,23787.071443,8519.777970,40013.878240
152,2025-05-04,24789.724557,8513.082863,40914.975649
153,2025-05-05,21990.858580,5620.771511,38713.847301
154,2025-05-06,20364.989988,3520.373095,35197.872399
155,2025-05-07,22479.683891,5452.520845,38566.666482


In [1]:
df = spark.read.table("mpesa_cleaned")
df.write.option("header", True).csv("Files/mpesa_cleaned_export")


StatementMeta(, 95c9f50f-5760-40bb-991b-c91fb69025cb, 3, Finished, Available, Finished)

In [2]:
# Read CSV from exported folder in OneLake
df = spark.read.csv("Files/mpesa_cleaned_export", header=True, inferSchema=True)

# Convert to Pandas to view and export from notebook
df_pd = df.toPandas()

# Show preview
df_pd.head()


StatementMeta(, 95c9f50f-5760-40bb-991b-c91fb69025cb, 4, Finished, Available, Finished)

,date,time,transaction_type,amount,direction,balance,narration
0,2024-10-09,2025-04-07 16:38:00,Salary Payment,3117.30,Out,11882.70,Salary Payment executed
1,2024-10-09,2025-04-07 09:34:00,Paybill Payment - Supplier,2577.35,Out,9305.35,Paybill Payment - Supplier executed
2,2024-10-09,2025-04-07 10:20:00,Customer Payment,12537.29,In,21842.64,Customer Payment executed
3,2024-10-09,2025-04-07 08:46:00,Loan Repayment,12538.96,Out,9303.68,Loan Repayment executed
4,2024-10-09,2025-04-07 18:58:00,Customer Payment,11545.49,In,20849.17,Customer Payment executed


In [3]:
df_pd.to_csv("Files/mpesa_cleaned_download.csv", index=False)



StatementMeta(, 95c9f50f-5760-40bb-991b-c91fb69025cb, 5, Finished, Available, Finished)

OSError: Cannot save file into a non-existent directory: 'Files'